In [ ]:
# Install praw: Python Reddit API Wrapper - to connect to Reddit
!pip install praw

# Install pandas: To store and manage data easily in tables
!pip install pandas

# Install nltk: Natural Language Toolkit - for doing Sentiment Analysis
!pip install nltk


In [ ]:
# Import praw to connect to Reddit
import praw

# Import pandas to create DataFrame
import pandas as pd

# Import nltk and the SentimentIntensityAnalyzer for text analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
# Download VADER's dictionary once
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import praw

reddit = praw.Reddit(
    client_id='4EATv3ukC1NyjlR5dYBm7w',
    client_secret='3vIXRROH7jgGwze3qr-2RcLJa9b4dQ',
    user_agent='safest_to_park by /u/HaarshaReddy',
    check_for_async=False
)

In [ ]:
print(reddit.read_only)


True


In [ ]:
# List of street names to check for parking safety
streets = [
    "Davis Ct", "Hamilton St", "Harrison Ave", "Hiram Pl",
    "Jersey St", "Kingsland Ave", "Supor Blvd", "Warren St",
    "6th St", "7th St", "Ann St", "Cleveland Ave", "Davis Ave"
]


In [ ]:
# Initialize VADER Sentiment Analyzer
sid = SentimentIntensityAnalyzer()


In [ ]:
# Create an empty list to store all results
results = []

# Start searching for each street
for street in streets:
    query = f"{street} Harrison NJ parking"  # Form the search query
    print(f"Searching for: {query}")          # For your information while running

    # Search in 'newjersey' subreddit (because Harrison is in NJ)
    for submission in reddit.subreddit('newjersey').search(query, limit=1000):  # Search 15 posts per street
        text = submission.title + " " + submission.selftext  # Combine title + post body

        # Apply Sentiment Analysis
        sentiment = sid.polarity_scores(text)
        compound = sentiment['compound']  # Overall feeling

        # Classify into Positive, Negative, or Neutral
        if compound >= 0.05:
            sentiment_label = "Positive"
        elif compound <= -0.05:
            sentiment_label = "Negative"
        else:
            sentiment_label = "Neutral"

        # Save the result in our list
        results.append({
            "street": street,
            "title": submission.title,
            "text": submission.selftext,
            "compound_score": compound,
            "sentiment": sentiment_label
        })

print("Finished scraping Reddit for all streets!")


Searching for: Davis Ct Harrison NJ parking
Searching for: Hamilton St Harrison NJ parking
Searching for: Harrison Ave Harrison NJ parking
Searching for: Hiram Pl Harrison NJ parking
Searching for: Jersey St Harrison NJ parking
Searching for: Kingsland Ave Harrison NJ parking
Searching for: Supor Blvd Harrison NJ parking
Searching for: Warren St Harrison NJ parking
Searching for: 6th St Harrison NJ parking
Searching for: 7th St Harrison NJ parking
Searching for: Ann St Harrison NJ parking
Searching for: Cleveland Ave Harrison NJ parking
Searching for: Davis Ave Harrison NJ parking
Finished scraping Reddit for all streets!


In [ ]:
# Make a nice DataFrame from results
df = pd.DataFrame(results)

# See the first few rows
df



,street,title,text,compound_score,sentiment
0,Davis Ct,"Harrison, NJ: Red Bull Arena 2hr Parking",Around Harrison are signs saying 2 hour parkin...,0.6705,Positive
1,Davis Ct,Has anyone ever had a wedding in a NJ park?,Hello! I am planning my wedding and want a sma...,0.8733,Positive
2,Davis Ct,NJ marijuana legalization AMA with Asbury Park...,"Hey, everyone. Mike Davis here. I, too, am rea...",-0.8360,Negative
3,Davis Ct,I hate when people say New Jersey is a terribl...,I know the meme is to not reveal the state's s...,0.9820,Positive
4,Davis Ct,Overnight/graveyard shift parking in Harrison ...,I have to commute to NYC 5 days a week for an ...,0.0000,Neutral
...,...,...,...,...,...
1481,Davis Ave,Can anyone help me find housing/apartment arou...,Were looking to get off campus for our senior ...,-0.6081,Negative
1482,Davis Ave,Places to find rent for working in Piscataway?,If working near [Old New Brunswick Rd and Cent...,0.9292,Positive
1483,Davis Ave,"Moving cross-country, how is the area in East/...","Hello I am moving to NJ, seeing a lot of cheap...",0.7818,Positive
1484,Davis Ave,Free Outdoor Movie Series in Monmouth County,We are just past the halfway point of our summ...,0.9564,Positive


In [ ]:
# prompt: i want to see this195 rows and columns so give me in excel

from google.colab import files
df.to_excel('reddit_sentiment_analysis.xlsx', index=False)
files.download('reddit_sentiment_analysis.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Group by street and sentiment
street_summary = df.groupby(['street', 'sentiment']).size().unstack(fill_value=0)

# Show the summary
print(street_summary)


sentiment      Negative  Neutral  Positive
street                                    
6th St               14       12        98
7th St               15       12        93
Ann St               15       12        90
Cleveland Ave        15       11        86
Davis Ave            17       12        89
Davis Ct              7        5        39
Hamilton St          17       17       126
Harrison Ave         15       11        85
Hiram Pl              2        4        37
Jersey St            42       31       163
Kingsland Ave        15       11        86
Supor Blvd            4        5        40
Warren St            14       14       105


In [ ]:
# Group the collected posts by street
street_post_counts = df['street'].value_counts()

# Show how many posts found for each street
print("Number of posts found per street:")
print(street_post_counts)


Number of posts found per street:
street
Jersey St        236
Hamilton St      160
Warren St        133
6th St           124
7th St           120
Davis Ave        118
Ann St           117
Cleveland Ave    112
Kingsland Ave    112
Harrison Ave     111
Davis Ct          51
Supor Blvd        49
Hiram Pl          43
Name: count, dtype: int64


In [ ]:
# Create a dictionary manually based on your table from picture
street_summary = {
    "6th St": {"Negative": 14, "Neutral": 12, "Positive": 98},
    "7th St": {"Negative": 15, "Neutral": 12, "Positive": 93},
    "Ann St": {"Negative": 15, "Neutral": 12, "Positive": 90},
    "Cleveland Ave": {"Negative": 15, "Neutral": 11, "Positive": 86},
    "Davis Ave": {"Negative": 17, "Neutral": 12, "Positive": 89},
    "Davis Ct": {"Negative": 7, "Neutral": 5, "Positive": 39},
    "Hamilton St": {"Negative": 17, "Neutral": 17, "Positive": 126},
    "Harrison Ave": {"Negative": 15, "Neutral": 11, "Positive": 85},
    "Hiram Pl": {"Negative": 2, "Neutral": 4, "Positive": 37},
    "Jersey St": {"Negative": 42, "Neutral": 31, "Positive": 163},
    "Kingsland Ave": {"Negative": 15, "Neutral": 11, "Positive": 86},
    "Supor Blvd": {"Negative": 4, "Neutral": 5, "Positive": 40},
    "Warren St": {"Negative": 14, "Neutral": 14, "Positive": 105}
}

# Create a new dictionary to store Left and Right sides
new_street_summary = {}

# For each street, create Left and Right with same data
for street, counts in street_summary.items():
    new_street_summary[street + " Left"] = counts
    new_street_summary[street + " Right"] = counts

# Convert to DataFrame
import pandas as pd

# Turn into a clean table
final_street_df = pd.DataFrame.from_dict(new_street_summary, orient='index')

# Show the final expanded table
print(final_street_df)


                     Negative  Neutral  Positive
6th St Left                14       12        98
6th St Right               14       12        98
7th St Left                15       12        93
7th St Right               15       12        93
Ann St Left                15       12        90
Ann St Right               15       12        90
Cleveland Ave Left         15       11        86
Cleveland Ave Right        15       11        86
Davis Ave Left             17       12        89
Davis Ave Right            17       12        89
Davis Ct Left               7        5        39
Davis Ct Right              7        5        39
Hamilton St Left           17       17       126
Hamilton St Right          17       17       126
Harrison Ave Left          15       11        85
Harrison Ave Right         15       11        85
Hiram Pl Left               2        4        37
Hiram Pl Right              2        4        37
Jersey St Left             42       31       163
Jersey St Right     

In [ ]:
# Make a fresh copy to work safely
rating_df_custom = final_street_df.copy()

# Define function based on new (4x, 3.5x, 3x, 2x) logic
def custom_rating_latest(row):
    negative = row['Negative']
    neutral = row['Neutral']
    positive = row['Positive']

    bad_score = negative + neutral

    if 4 * bad_score < positive:
        return 5  # 5 star
    elif 3.5 * bad_score < positive:
        return 4  # 4 star
    elif 3 * bad_score < positive:
        return 3  # 3 star
    elif 2 * bad_score < positive:
        return 2  # 2 star
    else:
        return 1  # 1 star

# Apply your new custom rating
rating_df_custom['Parking_Rating_Custom'] = rating_df_custom.apply(custom_rating_latest, axis=1)

# Show final result
print(rating_df_custom[['Negative', 'Neutral', 'Positive', 'Parking_Rating_Custom']])


                     Negative  Neutral  Positive  Parking_Rating_Custom
6th St Left                14       12        98                      4
6th St Right               14       12        98                      4
7th St Left                15       12        93                      3
7th St Right               15       12        93                      3
Ann St Left                15       12        90                      3
Ann St Right               15       12        90                      3
Cleveland Ave Left         15       11        86                      3
Cleveland Ave Right        15       11        86                      3
Davis Ave Left             17       12        89                      3
Davis Ave Right            17       12        89                      3
Davis Ct Left               7        5        39                      3
Davis Ct Right              7        5        39                      3
Hamilton St Left           17       17       126                

In [ ]:
# Step 1: Import libraries
import pandas as pd

# Step 2: Load your original CSV file
original_df = pd.read_csv('/content/harrison_parking_corrected_with_date_dataset.csv')  # Your uploaded original file

# Step 3: Create a dictionary from your picture values
new_data = {
    "6th St Left":  {"Negative": 14, "Neutral": 12, "Positive": 98, "Parking_Rating_Custom": 4},
    "6th St Right": {"Negative": 14, "Neutral": 12, "Positive": 98, "Parking_Rating_Custom": 4},
    "7th St Left":  {"Negative": 15, "Neutral": 12, "Positive": 93, "Parking_Rating_Custom": 3},
    "7th St Right": {"Negative": 15, "Neutral": 12, "Positive": 93, "Parking_Rating_Custom": 3},
    "Ann St Left":  {"Negative": 15, "Neutral": 12, "Positive": 90, "Parking_Rating_Custom": 3},
    "Ann St Right": {"Negative": 15, "Neutral": 12, "Positive": 90, "Parking_Rating_Custom": 3},
    "Cleveland Ave Left":  {"Negative": 15, "Neutral": 11, "Positive": 86, "Parking_Rating_Custom": 3},
    "Cleveland Ave Right": {"Negative": 15, "Neutral": 11, "Positive": 86, "Parking_Rating_Custom": 3},
    "Davis Ave Left": {"Negative": 17, "Neutral": 12, "Positive": 89, "Parking_Rating_Custom": 3},
    "Davis Ave Right": {"Negative": 17, "Neutral": 12, "Positive": 89, "Parking_Rating_Custom": 3},
    "Davis Ct Left": {"Negative": 7, "Neutral": 5, "Positive": 39, "Parking_Rating_Custom": 3},
    "Davis Ct Right": {"Negative": 7, "Neutral": 5, "Positive": 39, "Parking_Rating_Custom": 3},
    "Hamilton St Left": {"Negative": 17, "Neutral": 17, "Positive": 126, "Parking_Rating_Custom": 4},
    "Hamilton St Right": {"Negative": 17, "Neutral": 17, "Positive": 126, "Parking_Rating_Custom": 4},
    "Harrison Ave Left": {"Negative": 15, "Neutral": 11, "Positive": 85, "Parking_Rating_Custom": 3},
    "Harrison Ave Right": {"Negative": 15, "Neutral": 11, "Positive": 85, "Parking_Rating_Custom": 3},
    "Hiram PI Left": {"Negative": 2, "Neutral": 4, "Positive": 37, "Parking_Rating_Custom": 5},
    "Hiram PI Right": {"Negative": 2, "Neutral": 4, "Positive": 37, "Parking_Rating_Custom": 5},
    "Jersey St Left": {"Negative": 42, "Neutral": 31, "Positive": 163, "Parking_Rating_Custom": 2},
    "Jersey St Right": {"Negative": 42, "Neutral": 31, "Positive": 163, "Parking_Rating_Custom": 2},
    "Kingsland Ave Left": {"Negative": 15, "Neutral": 11, "Positive": 86, "Parking_Rating_Custom": 3},
    "Kingsland Ave Right": {"Negative": 15, "Neutral": 11, "Positive": 86, "Parking_Rating_Custom": 3},
    "Supor Blvd Left": {"Negative": 4, "Neutral": 5, "Positive": 40, "Parking_Rating_Custom": 5},
    "Supor Blvd Right": {"Negative": 4, "Neutral": 5, "Positive": 40, "Parking_Rating_Custom": 5},
    "Warren St Left": {"Negative": 14, "Neutral": 14, "Positive": 105, "Parking_Rating_Custom": 4},
    "Warren St Right": {"Negative": 14, "Neutral": 14, "Positive": 105, "Parking_Rating_Custom": 4}
}

# Step 4: Convert the dictionary to a DataFrame
new_data_df = pd.DataFrame.from_dict(new_data, orient='index')
new_data_df.index.name = 'Street Name'  # Match the original CSV column
new_data_df.reset_index(inplace=True)

# Step 5: Merge the old and new datasets based on "Street Name"
final_df = original_df.merge(new_data_df, on='Street Name', how='left')

# Step 6: Save the updated dataset into a new CSV in current folder
final_df.to_csv('final_parking_with_ratings.csv', index=False)

print("✅ Done! New file 'final_parking_with_ratings.csv' created successfully.")

In [ ]:
from google.colab import files
files.download('final_parking_with_ratings.csv')